In [52]:
import requests
import re
import pandas
from SPARQLWrapper import SPARQLWrapper, JSON

In [53]:
urla = "http://bridgedb.prod.openrisknet.org/Human/xrefs/Ca/"
urlb = "?dataSource=Wikidata"
urlc = "?dataSource=ChEBI"
tggatesconvert = 'http://open-tggates-api.cloud.douglasconnect.com/v2/'

In [54]:
CAS = ["103-90-2",
"88-18-6",
"96-76-4",
"98-54-4",
"105-67-9",
"106-44-5",
"128-37-0",
"128-39-2",
"140-66-9",
"576-26-1",
"697-82-5",
"2416-94-6",
"4130-42-1",
"120-80-9",
"123-31-9",
"700-13-0",
"1948-33-0",
"108-46-3",
"108-73-6",
"108-95-2",]

In [55]:
Phenols = {}
cas_wikidata = {}

In [56]:

for cas_id in CAS:
    cas_wikidata.update({cas_id:[requests.get(urla + cas_id + urlb).text.split("\t")[0]]})
print(cas_wikidata)

{'123-31-9': ['Q419164'], '98-54-4': ['Q2306557'], '108-95-2': ['Q130336'], '108-73-6': ['Q899008'], '128-37-0': ['Q221945'], '106-44-5': ['Q312251'], '2416-94-6': ['Q27236151'], '1948-33-0': ['Q662443'], '697-82-5': ['Q27117905'], '140-66-9': ['Q15632771'], '108-46-3': ['Q408865'], '105-67-9': ['Q2437536'], '700-13-0': ['Q27279264'], '4130-42-1': ['Q26841169'], '96-76-4': ['Q26840829'], '120-80-9': ['Q282440'], '88-18-6': ['Q27115979'], '128-39-2': ['Q2218184'], '576-26-1': ['Q1055852']}


In [57]:
def iter_cas(cas):
    uitkomst = ""
    for i in cas:
        uitkomst += '?CAS_nummer="' + i + '" || '
    return uitkomst[:-4]

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

pathwayQuery = '''SELECT ?item ?itemLabel ?CAS_nummer WHERE {
  ?item wdt:P231 ?CAS_nummer FILTER('''+ iter_cas(CAS) + ''').
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}'''



sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    cas_wikidata[result['CAS_nummer']['value']] += [result['itemLabel']['value']]
print(cas_wikidata)

{'123-31-9': ['Q419164', 'hydroquinone'], '98-54-4': ['Q2306557', '4-tert-butylphenol'], '108-95-2': ['Q130336', 'phenol'], '108-73-6': ['Q899008', 'phloroglucinol'], '128-37-0': ['Q221945', 'butylated hydroxytoluene'], '106-44-5': ['Q312251', 'para-cresol'], '2416-94-6': ['Q27236151', '2,3,6-trimethylphenol'], '1948-33-0': ['Q662443', 'tert-butylhydroquinone'], '697-82-5': ['Q27117905', '2,3,5-trimethylphenol'], '140-66-9': ['Q15632771', '4-tert-octylphenol'], '108-46-3': ['Q408865', 'resorcin'], '105-67-9': ['Q2437536', '2,4-Dimethylphenol'], '700-13-0': ['Q27279264', 'trimethylhydroquinone'], '4130-42-1': ['Q26841169', '2,6-di-tert-butyl-4-ethylphenol'], '96-76-4': ['Q26840829', '2,4-di-tert-butylphenol'], '120-80-9': ['Q282440', 'pyrocatechol'], '88-18-6': ['Q27115979', '2-tert-butylphenol'], '128-39-2': ['Q2218184', '2,6-di-tert-butylphenol'], '576-26-1': ['Q1055852', '2,6-Xylenol']}


In [58]:
compounds_name = ''
for names in cas_wikidata.values():
    compounds_name += ' | '+ names[1]

r2 = requests.get(tggatesconvert + 'samples',
                  params={'limit': 10000, 'compoundNameFilter': compounds_name[3:],
                          'organismFilter': 'Human', 'tissueFilter': 'Liver',
                          'cellTypeFilter': 'in vitro', 'repeatTypeFilter': 'Single',
                          'timepointHrFilter': '24.0', 'doseLevelFilter': 'High'
                          })


for x in r2.json()['aggregations']['compoundName']['buckets']:
    print(x['key'])

2,4-dinitrophenol
2-nitrofluorene
LPS
N-methyl-N-nitrosourea
N-nitrosomorpholine
TNFalpha
WY-14643
acarbose
acetamide
acetamidofluorene
acetaminophen
acetazolamide
adapin
aflatoxin B1
ajmaline
allopurinol
allyl alcohol
alpidem
amiodarone
amitriptyline
amphotericin B
aspirin
azathioprine
bendazac
benzbromarone
benziodarone
bromobenzene
bromoethylamine
bucetin
buspirone
buthionine sulfoximine
butylated hydroxyanisole
caffeine
captopril
carbamazepine
carbon tetrachloride
chloramphenicol
chlormadinone
chlormezanone
chlorpheniramine
chlorpromazine
chlorpropamide
cimetidine
ciprofloxacin
clofibrate
clomipramine
clozapine
colchicine
coumarin
cycloheximide
cyclophosphamide
cyclosporine A
danazol
dantrolene
dexamethasone
diazepam
diclofenac
diethyl maleate
diltiazem
disopyramide
disulfiram
doxorubicin
enalapril
erythromycin ethylsuccinate
ethambutol
ethanol
ethinylestradiol
ethionamide
ethionine
etoposide
famotidine
fenofibrate
fluoxetine hydrochloride
fluphenazine
flutamide
furosemide
galactos

In [59]:
print(len(cas_wikidata))
print(len(r2.json()['aggregations']['compoundName']['buckets']))

19
155
